# Preprocessamento S4
___

Este notebook realiza um pré-processamento dos dados de S4 disponibilizados em data_s4. O preprocessamento consiste de uma conversão de indices para tratamento de séries temporais, uma reamostragem para amostras de 1s (nos dados orginais, o número de medidas a cada segundo, dependendo do número de satélites visíveis, porém, em geral é mais que um), interpolaçãos spline, seguido de suavização com um kernel gaussiano com janela de tamanho 25, seguindo como uma reamostragem para dados a cada 10 mim, e uma suavização usando um kernel gausiano de janela 25.

In [1]:
import os
import re
import sys

import pathos.multiprocessing as mulprocessing
import pandas as pd

from functools import partial
from scipy.signal import savgol_filter
from pathos.multiprocessing import ProcessPool

import utils
from utils import local_s4, local_s4_pre
from utils import window

In [2]:
files = os.listdir(local_s4)

In [3]:
print(files)

['df_1.pkl.xz', 'df_11.pkl.xz', 'df_14.pkl.xz', 'df_15.pkl.xz', 'df_16.pkl.xz', 'df_17.pkl.xz', 'df_18.pkl.xz', 'df_19.pkl.xz', 'df_20.pkl.xz', 'df_22.pkl.xz', 'df_24.pkl.xz', 'df_25.pkl.xz', 'df_26.pkl.xz', 'df_28.pkl.xz', 'df_29.pkl.xz', 'df_3.pkl.xz', 'df_30.pkl.xz', 'df_31.pkl.xz', 'df_32.pkl.xz', 'df_33.pkl.xz', 'df_34.pkl.xz', 'df_4.pkl.xz', 'df_5.pkl.xz', 'df_6.pkl.xz', 'df_8.pkl.xz', 'df_afl.pkl.xz', 'df_bhz.pkl.xz', 'df_bov.pkl.xz', 'df_bsa.pkl.xz', 'df_cpa.pkl.xz', 'df_cub.pkl.xz', 'df_dou.pkl.xz', 'df_imp.pkl.xz', 'df_ios.pkl.xz', 'df_nta.pkl.xz', 'df_pln.pkl.xz', 'df_pvh.pkl.xz', 'df_sj2.pkl.xz', 'df_sta.pkl.xz', 'df_tfe.pkl.xz']


Inicialmente, os dados de S4 são tais que existe mais de uma amostra por minuto, decorrente dos múltiplos satélites de GPS que estavam no campo de visada da estão naquele intervalo.

O primeiro passo, assim, é combinar as várias medidas no intervalo de minuto gerando uma única medida por minuto, neste caso, tomou-se o valor m?dio de todos as medidas neste intervalo.

O segundo passo é tratar as entradas sem valores, aqui, decidiu-se por utilizar uma interpolação por spline de ordem 3.

O terceiro passo foi uma reamostragem dos dados para intervalores de 10 minutos, de forma, a compatibilizar com as taxas de amostragem dos demais dados a serem utilizados como o VTEC.

Segue-se então como uma suavização do sinal, por meio de uma aplicação de uma média móvel centrada com pesos gaussianos. Note, uma vez que trata-se de uma m?dia centrada:

\begin{equation}
y_t = \sum_{i=-n}^{+n}w_iy_{t+i}\mbox{,}
\end{equation}

onde $n$  a srepresenta o tamanho da janela. A suavização definiada, assim, leva em conta pontos futuros, tornando-a inadequada para aplicações em tempo real, pontos no futuro não são conhecidos, e em previsões de séries temporais, a suavização adiciona contribuição do futuro aos dados.

In [4]:
def preprocessing(df_file, window):
    df_s4 = pd.read_pickle(os.path.join(local_s4, df_file), compression='xz')
    df_s4['eventdate'] = pd.to_datetime(df_s4['eventdate'], utc=True)
    df_s4.index = df_s4['eventdate']
    del df_s4['eventdate']
    
    # change for 1s
    df_s4 = df_s4.resample('1T').mean()
    df_s4 = df_s4.interpolate(method='spline', order=4) #4

    # change for 10min
    df_s4 = df_s4.resample('10T').mean()

    # aplay filter Savitzky-Golay
    # aplay a gaussian move average
    df_s4['s4'] = savgol_filter(df_s4.values.ravel(), window, 3)
    df_s4 = df_s4.rolling(window, win_type='gaussian', center=True).mean(std=1.0)
    
    return df_s4

preprocessing_window = partial(preprocessing, window=window)

In [5]:
%%time
df_s4_list = ProcessPool(nodes=mulprocessing.cpu_count()).map(preprocessing_window, files)

CPU times: user 143 ms, sys: 57.3 ms, total: 200 ms
Wall time: 2min 14s


In [6]:
for name, df_s4 in zip(files, df_s4_list):
    df_s4.to_pickle(os.path.join(local_s4_pre,'pre_' + name.replace('.xz', '')))